In [1]:
import geopandas as gpd
import pypsa
import pandas as pd
import numpy as np
import os

Error loading 'pyutilib.component' entry points: 'The 'nose' distribution was not found and is required by the application'


In [2]:
years = np.arange(2020,2055,5)
scenarios = ['go','wait']
version = 'Base'

if version == 'w_Tran_exp':
    transmission = 'opt'
else:
    transmission = 'TYNDP'

In [3]:
def net_present_value(df): 
    rate = 0.02 # inflation rate
    npv = pd.Series()
    for scenario in scenarios:
        s = df[scenario].sum().droplevel(0)
        s.index = years
        s_n = s/(1+rate)**(s.index-2020)
        npv[scenario] = s_n.sum()*5/1e9
    return npv

In [4]:
df = pd.read_csv('../results/summaries/version-{}/csvs/costs.csv'.format(version),index_col=[0,1,2],header=[0,1,2],)
npv = net_present_value(df)

In [5]:
npv/npv['go']

go      1.000000
wait    1.041237
dtype: float64

In [6]:
d_n = {}
for scenario in scenarios:
    for year in years:
        d_n[(scenario,year)] = pypsa.Network('../results/networks/version-{}/postnetworks/postnetwork-{}_{}_{}.nc'.format(version,scenario,transmission,year))

Importing PyPSA from older version of PyPSA than current version 0.14.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network postnetwork-go_TYNDP_2020.nc has buses, carriers, generators, global_constraints, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version 0.14.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network postnetwork-go_TYNDP_2025.nc has buses, carriers, generators, global_constraints, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version 0.14.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network postnetwork-go_TYNDP_2030.nc has buses, carriers, generators, global_constraints, links, 

In [7]:
gen_carriers = ['onwind','offwind','solar','ror']
store_carriers = ['coal','oil','lignite','nuclear','gas']
su_carriers = ['hydro']

In [8]:
n = d_n[('go',2050)]
countries = n.buses.index[n.buses.carrier == 'AC'].tolist()
iterables = [countries, years]
columns = pd.MultiIndex.from_product(iterables, names=['country', 'year'])

# 2020 existing power capacity

In [9]:
costs = pd.read_csv('../data/costs/costs_2020.csv',index_col=list(range(2))).sort_index()
costs = costs.loc[:, "value"].unstack(level=1).groupby("technology").sum()
lf = costs.lifetime
eff = costs.efficiency

In [10]:
n = d_n[('go',2020)]

In [11]:
df = pd.read_csv('../data/existing_2020/existing_pp_2020.csv',index_col=0,dtype={'Fueltype':str,'Country':str})

# renewable potential

In [12]:
carriers = ['onwind','offwind','solar'] #,'biomass']
d_rp = {}

In [13]:
for carrier in carriers:
    df = pd.DataFrame(index=countries,columns=years)
    for year in years:
        n = d_n[('go',year)]
        df_ = n.generators[n.generators.carrier == carrier]
        df_ = df_.rename(lambda x: x[:2]).groupby(level=0).sum()
        df[year] = df_.p_nom_opt/df_.p_nom_max
    df = df.round(2)
    df.to_csv('results/{}_deployment_rate.csv'.format(carrier))
    d_rp[carrier] = df

# primary energy

In [14]:
def primary_energy(scenario,country):
    df = pd.DataFrame(index=gen_carriers+store_carriers+su_carriers,columns=years)

    for year in years:
        n = d_n[(scenario,year)]
        for carrier in gen_carriers:
            df.loc[carrier,year] = n.generators_t.p.sum().filter(like=country).filter(like=carrier).sum()
        for carrier in store_carriers:
            s = n.stores_t.e[country+' '+carrier+' store']
            df.loc[carrier,year] = s.iloc[0]-s.iloc[-1]
            
        carrier = 'hydro'
        try:
            df.loc[carrier,year] = n.storage_units_t.inflow.sum().filter(like=country).filter(like=carrier).item()
        except ValueError:
            df.loc[carrier,year] = 0
    df /= 1e6 #MWh to TWh
    return df

In [15]:
scenario = 'go'
df = pd.DataFrame(index=gen_carriers+store_carriers+su_carriers,columns=columns)
for country in countries:
    df[country] = primary_energy(scenario,country)

df.loc['hydro'] += df.loc['ror']
df.drop(index='ror',inplace=True)

df.to_csv('results/primary_energy_{}_{}.csv'.format(version,scenario))

# electricity production

In [16]:
def electricity_production(scenario,country):
    df = pd.DataFrame(index=gen_carriers+store_carriers+su_carriers,columns=years)

    for year in years:
        n = d_n[(scenario,year)]
        for carrier in gen_carriers:
            df.loc[carrier,year] = n.generators_t.p.sum().filter(like=country).filter(like=carrier).sum()
        for carrier in store_carriers:
            index = n.links.index[(n.links.bus0 == country+' '+carrier) & (n.links.bus1 == country)]
            df.loc[carrier,year] = -n.links_t.p1[index].sum().sum()
            
        carrier = 'hydro'
        try:
            df.loc[carrier,year] = n.storage_units_t.inflow.sum().filter(like=country).filter(like=carrier).item()
        except ValueError:
            df.loc[carrier,year] = 0
    df /= 1e6 #MWh to TWh
    return df

In [17]:
df = pd.DataFrame(index=gen_carriers+store_carriers+su_carriers,columns=columns)
for country in countries:
    df[country] = electricity_production(scenario,country)

df.loc['hydro'] += df.loc['ror']
df.drop(index='ror',inplace=True)

df.to_csv('results/electricity_production_{}_{}.csv'.format(version,scenario))

# heat production

In [18]:
heat_supply = ['gas boiler','CHP heat','resistive heater','heat pump']
iterables = [years, countries]
columns = pd.MultiIndex.from_product(iterables, names=['year','country'])
df = pd.DataFrame(index=heat_supply,columns=columns)

In [19]:
for year in years:
    n = d_n[(scenario,year)]
    for carrier in heat_supply:
        s = n.links.filter(regex=carrier,axis=0)
        hp = -n.links_t.p1[s.index].sum().rename(lambda x:x[:2]).groupby(level=0).sum()/1e6
        df.loc[carrier,year][hp.index] = hp.values
df.fillna(0,inplace=True)
df = df.reorder_levels(['country','year'],axis=1)

df.to_csv('results/heat_production_{}_{}.csv'.format(version,scenario))

In [20]:
store = ['battery','H2','water tank']
iterables = [years, countries]
columns = pd.MultiIndex.from_product(iterables, names=['year','country'])
df = pd.DataFrame(index=store,columns=columns)

In [21]:
for year in years:
    n = d_n[(scenario,year)]
    for carrier in store:
        s = n.stores.e_nom_opt.filter(like=carrier).rename(lambda x:x[:2]).groupby(level=0).sum()
        df.loc[carrier,year][s.index] = s.values
df.fillna(0,inplace=True)
df = df.reorder_levels(['country','year'],axis=1)

# df.to_csv('results/heat_production_{}_{}.csv'.format(version,scenario))

# revenue

In [36]:
iterables = [scenarios,years]
columns = pd.MultiIndex.from_product(iterables, names=['scenario','year'])
df = pd.DataFrame(columns=columns)

for scenario in scenarios:
    for year in years:
        n = d_n[(scenario,year)]
        p1 = -n.links_t.p1[n.links.index[n.links.p_min_pu != -1]]
        nmp = n.buses_t.marginal_price.reindex(columns=p1.columns.map(n.links.bus1))
        s_rev = (p1*nmp.values).sum()
        s_rev.index = s_rev.index.str[3:]
        s_rev = s_rev.groupby(level=0).sum()
        df[(scenario,year)] = s_rev
df = df.filter(regex='gas boiler|gas CHP',axis=0)    
df.to_csv('results/revenue_{}.csv'.format(version))